In [1]:
import numpy as np
import pandas as pd

import os
import sys

from sklearnex import patch_sklearn
patch_sklearn()

import sklearn

import sklearn.pipeline 
import sklearn.model_selection
import sklearn.metrics
import sklearn.linear_model

from sklearn.preprocessing import StandardScaler

import optuna

sys.path.append("/data/ouga/home/ag_gagneur/l_vilov/workspace/species-aware-DNA-LM/mpra_griesemer/utils") 

from models import *
from misc import dotdict

import scipy.stats

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/siegel_2022/'

In [28]:
input_params = dotdict({})

input_params.cell_type = 'Jurkat' #Beas2B or Jurkat
input_params.response = 'stability' #response variable: steady_state or stability

input_params.model = '5mers' #embedding name, can be "MLM" "word2vec" "griesemer" or "Nmers" where N is an integer

input_params.output_dir = './test' #output folder

input_params.N_trials = 1000 #number of optuna trials
input_params.keep_first = True #perform hpp search only at the first split, then use these hyperparameters

input_params.N_splits = 100 #number of GroupShuffleSplits
input_params.N_CVsplits = 5 #number of CV splits for hyperparameter search
input_params.seed = 1 #seed fot GroupShuffleSplit

In [29]:
mpra_df = pd.read_csv(data_dir + f'{input_params.cell_type}.tsv', sep='\t') #sequence info

mlm_embeddings = np.load(data_dir + "embeddings/seq_len_5000/embeddings.npy") #masked language model embeddings

In [30]:
if input_params.response == 'steady_state':
    mpra_df['Expression'] = mpra_df.ratios_T0_GC_resid
elif input_params.response == 'stability':
    mpra_df['Expression'] = mpra_df.ratios_T4T0_GC_resid

In [31]:
mpra_df.drop_duplicates(inplace=True)

flt = ~mpra_df.issnp.astype(bool) | mpra_df.Expression.isna()

#flt = mpra_df.Expression.isna()

mpra_df = mpra_df[~flt]

mlm_embeddings = mlm_embeddings[mpra_df.index]

In [32]:
mpra_df['group'] = mpra_df.region.apply(lambda x:x.split('|')[1].split(':')[0])

In [33]:
if input_params.model=='MLM':

    X = mlm_embeddings

elif 'mers' in input_params.model:
    
    k = int(input_params.model[0])
        
    kmerizer = Kmerizer(k=k)
    X = np.stack(mpra_df.seq.apply(lambda x: kmerizer.kmerize(x))) 
        
elif input_params.model=='word2vec':
        
    X = word2vec_model(mpra_df)

elif input_params.model=='griesemer':
        
    X = minseq_model(mpra_df)

#X = np.hstack((X,np.expand_dims(mpra_df.min_free_energy.values,axis=1)))

y = mpra_df['Expression'].values
groups = mpra_df['group'].values

In [34]:
def hpp_search(X,y,groups,cv_splits = 5):
    
    '''
    Perform Hyperparameter Search using OPTUNA Bayesian Optimisation strategy
    
    The bets hyperparameters should maximize coefficient of determination (R2)
    
    The hyperparameter range should first be adjused with grid search to make the BO algorithm converge in reasonable time
    '''

    def objective(trial):

        C = trial.suggest_float("C", 1e-2, 1e2, log=True)
        epsilon = trial.suggest_float("epsilon", 1e-5, 1, log=True)
        gamma = trial.suggest_float("gamma", 1e-5, 1, log=True)

        clf = sklearn.svm.SVR(C=C, epsilon=epsilon, gamma=gamma)

        pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),clf)

        cv_score = sklearn.model_selection.cross_val_score(pipe, X, y, groups=groups, 
                     cv = sklearn.model_selection.GroupKFold(n_splits = cv_splits), scoring = 'r2', n_jobs = -1)
        
        av_score = cv_score.mean()
        
        return av_score
    
    study = optuna.create_study(direction = "maximize")

    study.optimize(objective, n_trials = input_params.N_trials)
    
    best_params = study.best_params
    
    return best_params

In [40]:
cv_res = np.zeros((input_params.N_splits,len(y))) #predictions for each point in each split
cv_res[:] = np.NaN 

cv_scores = [] #scores and best hyperparameters for each split

#gss = sklearn.model_selection.LeaveOneGroupOut() 

input_params.N_splits=10

gss = sklearn.model_selection.GroupShuffleSplit(n_splits=input_params.N_splits, train_size=.9, random_state = input_params.seed) 
 

best_hpp = {'C': 0.12227907014412719, 'epsilon': 0.002017028863020407, 'gamma': 0.0017418739573905068}#MLM
best_hpp = {'C': 0.06074046127798098, 'epsilon': 0.032758360822165884, 'gamma': 0.0005466435051403621} #5mers

#best_hpp = {'alpha':1e-5}

for round_idx, (train_idx, test_idx) in enumerate(gss.split(X, y, groups)):
        
        X_train, X_test, y_train, y_test = X[train_idx,:],X[test_idx,:],y[train_idx],y[test_idx]
        
        #if round_idx==0 or input_params.keep_first==False:
            #perform only ones if input_params.keep_first==True
#            best_hpp = hpp_search(X_train,y_train,groups[train_idx],cv_splits = input_params.N_CVsplits)
        
        pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),
                                              sklearn.svm.SVR(**best_hpp))
        
        #pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(),
        #                                      sklearn.linear_model.Lasso(**best_hpp))
        
        pipe.fit(X_train,y_train)
                    
        y_pred = pipe.predict(X_test) 
        
        cv_res[round_idx,test_idx] = y_pred
        
        cv_scores.append({'round':round_idx,'r2':sklearn.metrics.r2_score(y_test,y_pred),
                         'pearson_r':scipy.stats.pearsonr(y_test,y_pred)[0]}|best_hpp)
        
cv_scores = pd.DataFrame(cv_scores)

In [41]:
cv_scores.pearson_r.mean()

0.4441137601189574

In [42]:
from sklearn.model_selection import cross_val_score

In [43]:
def pearson_r(estimator, X, y):
    y_pred = estimator.predict(X)
    #print(estimator[1].alpha_)
    if len(y_pred.shape) == 2:
        y_pred = y_pred.reshape(-1)
    return scipy.stats.pearsonr(y, y_pred)[0]

In [ ]:
pipe = sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(), sklearn.linear_model.LassoCV(cv=3, alphas=10.**np.arange(-6,0))) 
#limit runtime
r2 = cross_val_score(pipe,X[train_idx],y[train_idx],scoring=pearson_r,
                     cv=sklearn.model_selection.GroupKFold(n_splits=10), groups=groups[train_idx], n_jobs=-1)#.mean()

In [45]:
np.mean(r2)

0.43289348120315135

In [ ]:
os.makedirs(input_params.output_dir, exist_ok=True) #make output dir

cv_scores.to_csv(input_params.output_dir + '/cv_scores.tsv', sep='\t', index=None) #save scores

with open(input_params.output_dir + '/cv_res.npy', 'wb') as f:
    np.save(f, cv_res) #save predictions at each round